In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [2]:
# 显示所有列
#pd.set_option('display.max_columns', None)
# 显示所有行
#pd.set_option('display.max_rows', None)
# 设置value的显示长度为100，默认为50
#pd.set_option('max_colwidth', 100)

#import config


LABEL = 'label'


In [3]:
df_train = pd.read_csv(r'G:\新秩序\比赛\商品需求训练集.csv')
df_train_order = pd.read_csv(r'G:\新秩序\比赛\商品月订单训练集.csv')
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['year'] = df_train['date'].dt.year
df_train['month'] = df_train['date'].dt.month
df_train = df_train.groupby(['product_id', 'year', 'month'])[['is_sale_day', 'label']].sum().reset_index()
df_train = df_train.merge(df_train_order, on=['product_id', 'year', 'month'], how='left')

df_test = pd.read_csv(r'G:\新秩序\比赛\商品需求测试集.csv')
df_test_order = pd.read_csv(r'G:\新秩序\比赛\商品月订单测试集.csv')
df_test['date'] = pd.to_datetime(df_test['date'])
df_test['year'] = df_test['date'].dt.year
df_test['month'] = df_test['date'].dt.month
df_test = df_test.groupby(['product_id', 'year', 'month'])[['is_sale_day']].sum().reset_index()
df_test = df_test.merge(df_test_order, on=['product_id', 'year', 'month'], how='left')

In [4]:
df = pd.concat([df_train, df_test])

for i in range(1, 8):
    for f in [LABEL, 'order', 'start_stock', 'end_stock']:
        df[f+'_shift_%d'%i] = df.groupby('product_id')[f].shift(i+3)

for i in range(1, 4):
    for f in ['order', 'start_stock', 'end_stock']:
        df[f+'_shift_-%d'%i] = df.groupby('product_id')[f].shift(-i)

for i in [3, 6]:
    for f in [LABEL, 'order', 'start_stock', 'end_stock']:
        df[f+'_mean_%d'%i] = df[[f+'_shift_%d'%i for i in range(1, i+1)]].mean(axis=1)
        df[f+'_std_%d'%i] = df[[f+'_shift_%d'%i for i in range(1, i+1)]].std(axis=1)
        df[f+'_median_%d'%i] = df[[f+'_shift_%d'%i for i in range(1, i+1)]].median(axis=1)
# df.to_csv(config.save_trans_data_dl_path, index=False)
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])
df['type'] = df['type'].astype('category')

df['product_id'] = df['product_id'].astype('category')

df_train = df[df[LABEL].notna()].reset_index(drop=True)
df_test = df[df[LABEL].isna()].reset_index(drop=True)
feats = [f for f in df_test if f not in ['year', 'label']]

df_train[LABEL+'_log1p'] = np.log1p(df_train[LABEL])

In [5]:
def trymodel(df_train,params):
    LABEL="label"
    seeds=[615]
    seed=615
    fold_num = 3
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    oof = np.zeros(len(df_train))
    pred_y = pd.DataFrame()
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train['product_id'])):
        train = lgb.Dataset(df_train.loc[train_idx, feats],df_train.loc[train_idx, LABEL+'_log1p'])
        val = lgb.Dataset(df_train.loc[val_idx, feats],df_train.loc[val_idx, LABEL+'_log1p'])
        model = lgb.train(params, train, valid_sets=[train, val], num_boost_round=1000,callbacks=[lgb.early_stopping(100)])
        
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        #importance += model.feature_importance(importance_type='gain') / fold_num
        
    df_train['target_weight'] = df_train[LABEL] / df_train.groupby(['year', 'month'])[LABEL].transform('sum')
    df_train['oof'] = np.expm1(oof)
    score1 = np.sum((1 - np.abs(df_train[LABEL]-df_train['oof']) / (df_train[LABEL])
                    ) *
                    df_train['target_weight']) / 35
    scores = score1
    print(scores)
    penis=np.mean(np.abs(df_train[LABEL]-np.expm1(oof)))
    print(np.mean(np.abs(df_train[LABEL]-np.expm1(oof))))
    
    return scores

In [6]:
import optuna

In [ ]:
def objective(trial):

    # 2. Suggest values of the hyperparameters using a trial object.
    params22 = {
        'metric': 'mse',
        #'learning_rate': 0.05,
        'objective':'regression_l1',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 1e-2, 1e-1, log=True),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    kkjb = trymodel(df_train,params22)
    
    return kkjb

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2022-07-17 14:44:46,186] A new study created in memory with name: no-name-d72012f7-ff13-44ee-bb74-ac555da7c7b0
G:\anaconda3\envs\woshidashabi\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
G:\anaconda3\envs\woshidashabi\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.61157	valid_1's l2: 0.578126
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.573676	valid_1's l2: 0.709944
Training until validation scores don't improve for 100 rounds


[I 2022-07-17 14:44:52,005] Trial 0 finished with value: 0.6521411884388811 and parameters: {'learning_rate': 0.03286101032945966, 'lambda_l1': 2.142365010719385e-08, 'lambda_l2': 1.254730129103374, 'num_leaves': 156, 'feature_fraction': 0.7646956316986999, 'bagging_fraction': 0.7772981869350989, 'bagging_freq': 1, 'min_child_samples': 89}. Best is trial 0 with value: 0.6521411884388811.


Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.600629	valid_1's l2: 0.678224
0.6521411884388811
601.2458421639263


G:\anaconda3\envs\woshidashabi\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
G:\anaconda3\envs\woshidashabi\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


...

Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.241085	valid_1's l2: 0.392574
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.2331	valid_1's l2: 0.472583
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.257386	valid_1's l2: 0.435431


[I 2022-07-17 14:55:59,381] Trial 42 finished with value: 0.7149859131543111 and parameters: {'learning_rate': 0.03615022186272043, 'lambda_l1': 4.3996814053273714e-08, 'lambda_l2': 0.059518781696041606, 'num_leaves': 182, 'feature_fraction': 0.64696658847195, 'bagging_fraction': 0.8080494262117324, 'bagging_freq': 1, 'min_child_samples': 14}. Best is trial 31 with value: 0.7269480013649422.


0.7149859131543111
507.47674295479067
Training until validation scores don't improve for 100 rounds


G:\anaconda3\envs\woshidashabi\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
G:\anaconda3\envs\woshidashabi\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.326616	valid_1's l2: 0.406237
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1000]	training's l2: 0.288071	valid_1's l2: 0.508707
Training until validation scores don't improve for 100 rounds
